In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler 

# Data Loading
mnist = pd.read_csv("./mnist_for_kaggle/train.csv")
num_of_train = int(mnist.shape[0] * 0.8)
train_data = mnist.loc[:num_of_train,:]
test_data = mnist.loc[num_of_train+1:,:]

train_x_data = train_data.drop("label", axis=1, inplace=False).values
train_y_data = tf.one_hot(train_data["label"].values,10)
sess = tf.Session()
train_y_data = sess.run(train_y_data)

test_x_data = test_data.drop("label", axis=1, inplace=False).values
test_y_data = tf.one_hot(test_data["label"].values,10)
test_y_data = sess.run(test_y_data)

# Mni_Max_Scaling
train_x_data = MinMaxScaler().fit_transform(train_x_data)
test_x_data = MinMaxScaler().fit_transform(test_x_data)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [15]:
# Model
tf.reset_default_graph()

# 2.1 placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
dropout_rate = tf.placeholder(dtype=tf.float32)

# 2.2 Convolution Layer
# 입력데이터는 4차원!
x_img = tf.reshape(X,[-1,28,28,1])
L1 = tf.layers.conv2d(inputs=x_img,filters=32,
                     kernel_size=[3,3], padding="SAME",
                     strides=1, activation=tf.nn.relu)
L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2],
                             padding="SAME",
                             strides=2)
L1 = tf.layers.dropout(inputs=L1, rate=dropout_rate)                             

L2 = tf.layers.conv2d(inputs=L1,filters=64,
                     kernel_size=[3,3], padding="SAME",
                     strides=1, activation=tf.nn.relu)
L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],
                             padding="SAME",
                             strides=2)
L2 = tf.layers.dropout(inputs=L2, rate=dropout_rate)

# 2.3 FC Layer => dense layer
L2 = tf.reshape(L2,[-1,7*7*64])

dense1 = tf.layers.dense(inputs=L2, units=256,
                        activation=tf.nn.relu)
dense1 = tf.layers.dropout(inputs=dense1, rate=dropout_rate)

dense2 = tf.layers.dense(inputs=dense1, units=128,
                        activation=tf.nn.relu)
dense2 = tf.layers.dropout(inputs=dense2, rate=dropout_rate)

dense3 = tf.layers.dense(inputs=dense2, units=512,
                        activation=tf.nn.relu)
dense3 = tf.layers.dropout(inputs=dense3, rate=dropout_rate)

H = tf.layers.dense(inputs=dense3, units=10)

# cost
cost = tf.losses.softmax_cross_entropy(Y,H)

# train
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

# session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# ensenble
num_of_models = 10
result_model = np.zeros([test_x_data.shape[0],10])

for models in range(num_of_models):
  # 학습
  num_of_epoch = 30
  batch_size = 100  

  for step in range(num_of_epoch):
    num_of_iter = int(train_x_data.shape[0] / batch_size)
    cost_val = 0
    idx = 0
    for i in range(num_of_iter):    
      batch_x = train_x_data[idx:idx+batch_size,:]
      batch_y = train_y_data[idx:idx+batch_size,:]
      idx = idx + batch_size  
      _, cost_val = sess.run([train,cost], feed_dict={X:batch_x,
                                                      Y:batch_y,
                                                      dropout_rate:0.4})
    if step % 3 == 0:
      print("cost : {}".format(cost_val))

  tmp = sess.run(H, feed_dict={X:test_x_data,
                               Y:test_y_data,
                               dropout_rate:0})
  result_model = result_model + tmp
  

# 정확도
predict = np.argmax(result_model,1)
correct = np.equal(predict, np.argmax(test_y_data,1))
accuracy = np.mean(correct)
print("정확도 : {}".format(accuracy))
# 99.2 %의 정확도를 가지는 모델임을 확인!!

# 마지막으로 prediction할때는
# 10개의 모델이 다 있고 각 모델의 예측값을 구해서
# 이 모델의 예측값의 합을 이용해서 최종 예측값을 구해야하는데!!
# 지금 모델이 다 날라가고..마지막 1개의 모델만 남아있는 형태!

# 어떻게 해야하나요??
# 각각의 모델을 어딘가에 저장!!
# 객체지향방식을 이용해서 처리하는게 가장 좋아요!

# 모듈화!
# 기능의 모듈화 => 자주사용되는 코드를 함수화 시켜서
#                  반복처리를 하는 것을 의미
# 객체지향의 모듈화는 기능 + 데이터를 함께 모듈화해요!








KeyboardInterrupt: ignored

In [0]:
#